In [1]:
!git clone 'https://github.com/crispitagorico/Signature-Kernel.git'

Cloning into 'Signature-Kernel'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 661 (delta 58), reused 62 (delta 30), pack-reused 570
Receiving objects: 100% (661/661), 22.67 MiB | 32.42 MiB/s, done.
Resolving deltas: 100% (360/360), done.


In [2]:
%cd /content/Signature-Kernel/src/ 

/content/Signature-Kernel/src


In [3]:
!python setup.py build_ext --inplace

Compiling sigKer_fast.pyx because it changed.
[1/1] Cythonizing sigKer_fast.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Signature-Kernel/src/sigKer_fast.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'sigKer_fast' extension
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c sigKer_fast.c -o build/temp.linux-x86_64-3.6/sigKer_fast.o
creating build/lib.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x8

In [4]:
import sys
sys.path.append('../src')
sys.path.append('../utils')

import torch
import time
import numpy as np

from sigLoss import SigLoss, SigLoss_naive
from sigMMD import SigMMD, SigMMD_naive

import matplotlib.pyplot as plt

# Sig Loss gradients

In [5]:
A = 7
M = 10
N = 12
D = 2

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(A,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

X_gpu = X_naive.clone().cuda()
Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
X_gpu.requires_grad = True

In [6]:
t = time.time()
l_naive = SigLoss_naive(n_chunks=1).forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(l_naive)

time: 0.083 s
tensor(1.2582, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [20]:
t = time.time()
l_cpu = SigLoss(n_chunks=1).forward(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(l_cpu)

time: 0.002 s
tensor(2.3462, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [21]:
t = time.time()
l_gpu = SigLoss(n_chunks=1).forward(X_gpu,Y_gpu)
print('time:', np.round(time.time()-t,3), 's')
print(l_gpu)

time: 0.225 s
tensor(2.3462, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)


In [22]:
t = time.time()
l_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.069 s


In [23]:
t = time.time()
l_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.003 s


In [24]:
t = time.time()
l_gpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.007 s


In [41]:
torch.allclose(X_naive.grad, X_cpu.grad)

True

In [42]:
torch.allclose(X_naive.grad, X_gpu.grad.cpu())

True

# Sig MMD gradients

In [27]:
A = 3
B = 2
M = 7
N = 6
D = 4

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(B,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

X_gpu = X_naive.clone().cuda()
Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
X_gpu.requires_grad = True

In [28]:
t = time.time()
mmd_naive = SigMMD_naive().forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_naive)

time: 0.028 s
tensor(1.7848, dtype=torch.float64, grad_fn=<SubBackward0>)


In [29]:
t = time.time()
mmd_cpu = SigMMD().forward(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_cpu)

time: 0.002 s
tensor(1.7848, dtype=torch.float64, grad_fn=<SubBackward0>)


In [30]:
t = time.time()
mmd_gpu = SigMMD().forward(X_gpu,Y_gpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_gpu)

time: 0.009 s
tensor(1.7848, device='cuda:0', dtype=torch.float64, grad_fn=<SubBackward0>)


In [31]:
t = time.time()
mmd_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.043 s


In [32]:
t = time.time()
mmd_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.004 s


In [33]:
t = time.time()
mmd_gpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.008 s


In [39]:
torch.allclose(X_naive.grad, X_cpu.grad)

True

In [40]:
torch.allclose(X_naive.grad, X_gpu.grad.cpu())

True